# Tensorflow estimators
## Define an Estimator - dataset

In [51]:
import numpy as np
import tensorflow as tf

x_train = np.linspace(0,1,100)[:,np.newaxis]
y_train = np.sin(x_train*6)

def dataset():
    return tf.data.Dataset.from_tensor_slices(
        {"x": x_train, "y": y_train}).batch(5)

In [54]:
next_element

{'x': <tf.Tensor 'IteratorGetNext_11:0' shape=(?, 1) dtype=float64>,
 'y': <tf.Tensor 'IteratorGetNext_11:1' shape=(?, 1) dtype=float64>}

In [53]:
iterator = dataset().make_one_shot_iterator()
next_element = iterator.get_next()
sess = tf.Session()
sess.run(next_element)

{'x': array([[0.        ],
        [0.01010101],
        [0.02020202],
        [0.03030303],
        [0.04040404]]), 'y': array([[0.        ],
        [0.06056897],
        [0.12091552],
        [0.18081808],
        [0.24005668]])}

## Define an Estimator - model function

In [57]:
def model_fn(features, labels, mode, params):
    x = features['x']
    y = features['y']
    hidden1 = tf.layers.dense(x, 8)
    hidden2 = tf.layers.dense(hidden1, 8)
    y_pred = tf.layers.dense(hidden2, 1)
    loss = tf.losses.mean_squared_error(y, y_pred)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)
        
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {"y": y}
        return tf.estimator.EstimatorSpec(
            mode, predictions=predictions)

## Simple neural network with Estimator

In [58]:
estimator = tf.estimator.Estimator(model_dir="simple_estimator", model_fn=model_fn)
estimator.train(input_fn=dataset, max_steps=10000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'simple_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7ec46a2e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph wa